In [7]:
# 基础代码引入
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

print(api_key)
print(base_url)

chat = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [1]:
from langchain.memory import ConversationBufferMemory

# 手动添加对话记录到memory
# 方式2: 
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [2]:
memory = ConversationBufferMemory()
# 手动添加对话记录到memory
# 方式2: input就是add_user_message，output就是add_ai_message
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [3]:
# 返回message格式
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi!'), AIMessage(content="what's up?")]}

In [4]:
memory.chat_memory.messages


[HumanMessage(content='hi!'), AIMessage(content="what's up?")]

In [5]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
# 手动添加对话记录到memory
# 方式3: 
history.add_user_message("hi!")

history.add_ai_message("whats up?")
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("你是谁!")
memory.chat_memory.add_ai_message("我是一个诗人")

In [10]:
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
    # 每次对话会把记录保存下来
    memory=memory
)
conversation({"question": "你是诗人吗"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:
Human: 你是谁!
AI: 我是一个诗人
Human: 你是诗人吗
AI: 是的，我是一位诗人，我们可以一起探讨诗歌和创作。

New human question: 你是诗人吗
Response:

> Finished chain.


{'question': '你是诗人吗',
 'chat_history': 'Human: 你是谁!\nAI: 我是一个诗人\nHuman: 你是诗人吗\nAI: 是的，我是一位诗人，我们可以一起探讨诗歌和创作。',
 'text': '是的，我是一个诗人，我们可以一起探讨诗歌和创作。'}

In [9]:
memory.load_memory_variables({})


{'chat_history': 'Human: 你是谁!\nAI: 我是一个诗人\nHuman: 你是诗人吗\nAI: 是的，我是一位诗人，我们可以一起探讨诗歌和创作。'}

In [ ]:
# memory 本身是一种简单的记忆方式，不具备推理的能力

In [1]:
from langchain.memory import ConversationKGMemory

# ConversationKGMemory 存储记录，并分析内容形成图谱
# KG：只是图谱
memory = ConversationKGMemory(llm=chat)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})
memory.load_memory_variables({"input": "who is sam"})

NameError: name 'chat' is not defined

In [ ]:
# 在chain中使用kg
from langchain.chains import ConversationChain

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=chat, verbose=True, prompt=prompt, memory=ConversationKGMemory(llm=chat)
)